In [2]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

import time
from datetime import datetime

from pandas import Series as Series
from pandas import DataFrame as DataFrame
import logging

In [3]:
chrome_path = 'C:/Users/jin/chromedriver_win32/chromedriver.exe'
option = Options()
option.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 1
})
driver = webdriver.Chrome(options=option, executable_path=chrome_path)

# Open "Google" page in parent window
driver.get("https://google.com")

driver.title # 'Google'

# Get parent window
parent_window = driver.current_window_handle 

# Open "Bing" page in child window
driver.execute_script("window.open('https://bing.com')") 

# Get list of all windows currently opened (parent + child)
all_windows = driver.window_handles 

# Get child window
child_window = [window for window in all_windows if window != parent_window][0] 

# Switch to child window
driver.switch_to.window(child_window) 

driver.title # 'Bing'

# Close child window
driver.close() 

# Switch back to parent window
driver.switch_to.window(parent_window) 

driver.title # 'Google'

'Google'

In [2]:
class JoongAngCrawler:
    def __init__(self, logger, start_meta):
        self.logger = logger

        # 드라이버 지정
        self.chrome_path = 'C:/Users/jin/chromedriver_win32/chromedriver.exe'
        self.option = Options()
        self.option.add_experimental_option("prefs", {
            "profile.default_content_setting_values.notifications": 1
        })
        self.driver = webdriver.Chrome(options=self.option, executable_path=self.chrome_path)
        self.url = "https://www.joongang.co.kr/society/accident"
        self.driver.get(self.url)

        self.start_meta = start_meta
        self.save_dir = './data/'

    def is_between(self, request: str, start: str, end: str) -> bool:
        """
        return: whether the request date is between start date and end date.
        """
        start = datetime.strptime(start, '%Y.%m.%d %H:%M')
        request = datetime.strptime(request, '%Y.%m.%d %H:%M')
        end = datetime.strptime(end, '%Y.%m.%d %H:%M')

        return end <= request <= start

    def is_before(self, request: str, base: str) -> bool:
        """
        return: whether the base date is the same as or immediately before the requested date.
        """
        request = datetime.strptime(request, '%Y.%m.%d %H:%M')
        base = datetime.strptime(base, '%Y.%m.%d %H:%M')

        return base <= request

    def finder(self, request_meta, page_num):
        print(f"{request_meta}의 기사를 찾으러 내려가겠습니다!")
        while True:
            # page_num = 0

            print ("로딩 중...")
            time.sleep(3)
            story_list = self.driver.find_element_by_css_selector("#story_list").find_elements_by_class_name("card_body")
            print("현재 페이지의 기사 수: ",len(story_list))

            # 24 articles in one page
            meta_start = self.driver.find_elements_by_class_name("date")[page_num*24] # 0
            meta_end = self.driver.find_elements_by_class_name("date")[(page_num+1)*24-1] # 23

            print(f"해당 범위에서 탐색 시작: {meta_start.text} ~ {meta_end.text}")

            if self.is_between(request_meta, meta_start.text, meta_end.text):
                for idx in range(page_num*24, (page_num+1)*24):
                    base_meta = story_list[idx].find_element_by_class_name("date")
                    if self.is_before(request_meta, base_meta.text):
                        # print(story_list[idx].find_element_by_class_name("headline").text)
                        # print(idx)
                        print("기사를 찾았습니다!")
                        return story_list[idx]
            else:
                print("현재 범위에서 찾고자 하는 날짜가 없습니다!")
                print("더 보기를 클릭합니다.")
                expand_button = self.driver.find_element_by_css_selector("#container > section > div > div.col_lg9 > section > div > a")
                # 버튼을 찾을 때까지 스크롤 이동
                # self.driver.execute_script('arguments[0].scrollIntoView(true);', expand_button)
                # self.driver.execute_script('window.scrollTo(0, 300);')
                WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#container > section > div > div.col_lg9 > section > div > a")))
                # 더보기 버튼 클릭
                expand_button.click()
                page_num+=1

    def crawling(self):
        self.logger.info("Crawler Start!")
        start_page = self.finder(self.start_meta, 0)
        print(start_page.find_element_by_class_name("headline").text)
        
        accident_df = DataFrame(columns=['date', 'title','body', 'URL'])
        accident_df.to_csv(self.save_dir+'joongang_accident_df')

In [3]:
if __name__=="__main__":
    # logger
    logger = logging.getLogger("log")
    logger.setLevel(logging.INFO)
    file_handler= logging.FileHandler('./log/')

    formatter = logging.Formatter('%(asctime)s|%(levelname)s: %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    crawler = JoongAngCrawler(logger,'2021.09.01 00:00')
    crawler.crawling()
    

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=93.0.4577.63)


In [8]:

import pandas as pd

save_dir = ('./data/joongang_accident_df.csv')
accident_df = pd.DataFrame(columns={'date', 'title', 'body', 'url'})
accident_df = accident_df[['date', 'title', 'body', 'url']]
accident_df.to_csv(save_dir, index=False)
df = pd.read_csv(save_dir)
print(df)

Empty DataFrame
Columns: [date, title, body, url]
Index: []


In [18]:
import pandas as pd
save_dir = ('./data/joongang_accident_df.csv')
df = pd.read_csv(save_dir)
print(len(df))
print(df.iloc[-1])

146
body     사진 해양경찰청\n의무경찰로 복무 중인 아들을 자신이 지휘하는 함정에서 근무하도록 ...
date                                      2021.06.27 21:29
url            https://www.joongang.co.kr/article/24092198
title                       자기 배에 의경아들 배치…해경함장 자택서 숨진 채 발견
Name: 145, dtype: object


In [ ]:
pd.concat([df1,df2], ignore_index=True)

In [ ]:
# 맨 처음 검색할 날짜 지정
date = "2020-12-31"

# 몇일동안 크롤링할지
#date_cnt = 0

# 수집한 기사 개수 카운트
article_cnt = 0

while (article_cnt < 2500):
    # 먼저 초기화하고 while문 안에서 page수 계산한 다음 그만큼 돈다
    p = 1
    pages = 1
    # 해당 날짜에 나온 기사의 여러 페이지 반복탐색
    while (p<=pages):
        url = "https://news.joins.com/society/accident/list/{}?filter=OnlyJoongang&Date={}".format(p, date)
        driver.get(url)
#         driver.maximize_window()
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pages = len(driver.find_elements_by_class_name("link_page"))
        
        # t_len: 한 페이지 안의 총 기사 개수
        t_len = len(driver.find_elements_by_css_selector("#content > div.list_basic > ul > li"))
        
        # 한 페이지 안의 기사 수 만큼 탐색
        for i in range(1, t_len+1):
            if t_len==1:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li > h2 > a")
            else:
                a = driver.find_element_by_css_selector("#content > div.list_basic > ul > li:nth-child({}) > h2 > a".format(i))
                
            # 제목 수집
            tit = a.text
            
            # 개별 기사 클릭
            a.send_keys(Keys.ENTER)
                
            # 본문 수집
            body = WebDriverWait(driver, 10).until(lambda x : x.find_element_by_css_selector("#article_body")).text
                
            # 데이터 프레임에 삽입
            accident_df.loc[article_cnt] = [date, tit, body, 3]
            article_cnt+=1
            
            print(date)
            print(tit)
            print("------------------------------------------------")
                
            # 2500개가 돼도 페이지 안의 기사 수 만큼 for문을 돌기 때문에 2500개일 때 break
            if article_cnt==2500:
                break
                
            driver.back()
            driver.implicitly_wait(3)
            

        p+=1
        if article_cnt==2500:
                break

    if article_cnt==2500:
                break
    # 이전 날짜로 가는 버튼 클릭
    action=ActionChains(driver)
    #h2 = WebDriverWait(driver,10).until(lambda x: x.find_element_by_css_selector("body > div.doc > header > div.mh > div > h2"))
    driver.execute_script("window.scrollTo(0, 0)")
    btn = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_prev")))
    #print(btn)
    btn.send_keys(Keys.ENTER)
    date = WebDriverWait(driver,10).until(lambda x : x.find_element_by_id("display_search_date")).text.replace(".","-")
    
    #date_cnt+=1

driver.close()

accident_df.to_csv(save_dir+'accident_df')